In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
target_date = pd.to_datetime("2022-09-01")
# parquet_file_path = "/home/knowit/Home_Foresee/forseeModel/data/VMS_DCA_joined/makrell_not.parquet"
parquet_file_path = (
    "/home/knowit/Home_Foresee/forseeModel/data/VMS_DCA_joined/cod_trawl.parquet"
)
df_long = pd.read_parquet(parquet_file_path)
df = df_long.groupby("route_id").last().reset_index()
# df = df[df['position_time'] <= target_date]
df = df.sort_values("position_time")

In [ ]:
df = df[["roundweight", "position_time", "lon", "lat"]]

In [ ]:
X = df[["position_time", "lon", "lat"]]
y = df["roundweight"]

In [ ]:
import xarray as xr

bio = xr.open_dataset("data/copernicus/nrt/resampled/bio_north_sea.nc")
bio

In [ ]:
import pyproj

t = pyproj.Transformer.from_crs("4326", "3035", always_xy=True)
it = pyproj.Transformer.from_crs("3035", "4326", always_xy=True)
crs_x, crs_y = t.transform(df["lon"], df["lat"])
df["x"] = crs_x
df["y"] = crs_y

In [ ]:
grid_x, grid_y = bio.x.values, bio.y.values
x_min, x_max = grid_x.min(), grid_x.max()
y_min, y_max = grid_y.min(), grid_y.max()
x_step = np.abs(grid_x[1] - grid_x[0])
y_step = np.abs(grid_y[1] - grid_y[0])

In [ ]:
df = df[df["x"].between(x_min, x_max)]
df = df[df["y"].between(y_min, y_max)]

In [ ]:
# Convert lon and lat to grid indices
x_idx = np.round((df["x"] - x_min) / x_step)
y_idx = np.round((df["y"] - y_min) / y_step)

In [ ]:
df["x_round"] = x_min + x_idx * x_step
df["y_round"] = y_min + y_idx * y_step
# df['t_round'] = df['position_time'].dt.month
df["t_round"] = df["position_time"].dt.normalize()
# df['t_round'] = np.round(df['position_time'].astype(int) / 86400)

In [ ]:
# df_round = df.groupby([pd.Grouper(key='t_round', freq='30D'), 'x_round', 'y_round']).agg({'roundweight': 'sum'}).reset_index()
df_round = (
    df.groupby(["t_round", "x_round", "y_round"])
    .agg({"roundweight": "sum"})
    .reset_index()
)
# df_round['t_round'].dt.month
df_round = (
    df_round.groupby([df_round["t_round"].dt.month, "x_round", "y_round"])
    .agg({"roundweight": "mean"})
    .reset_index()
)
# df_round = df_round.groupby(pd.Grouper(key='t_round', freq='30D')).agg({'roundweight': 'sum'}).reset_index()
df_round = df_round.sort_values("t_round")
df_round

In [ ]:
df_long["month"] = df_long["position_time"].dt.month
in_grid = df_long["start_lon"].between(-3, 10) & df_long["start_lat"].between(55, 65)
route_ids = (
    df_long[in_grid]
    .groupby(["route_id", "month"])
    .size()
    .sort_values(ascending=False)
    .reset_index()
    .groupby("month")
    .head(10)
)  # ['route_id']
# route_ids = df_long[in_grid].groupby(['route_id', 'month']).first().sort_values(by='roundweight', ascending=False).reset_index().groupby('month').head(10)#['route_id']
route_ids

In [ ]:
vmin = np.quantile(df_round["roundweight"], 0.0)
vmax = np.quantile(df_round["roundweight"], 0.9)

In [ ]:
import cartopy.crs as ccrs

fig = plt.figure(figsize=(16, 20))
for i, (month, data) in enumerate(df_round.groupby("t_round")):
    print(month)
    ax = plt.subplot(3, 4, i + 1, projection=ccrs.epsg("3035"))
    ax.stock_img()
    ax.scatter(
        data.x_round, data.y_round, c=data.roundweight, s=5, vmin=vmin, vmax=vmax
    )
    # ax.set_extent([x_min, x_max, y_min, y_max], crs=ccrs.epsg('3035'))

    top_route_ids = route_ids.loc[route_ids["month"] == month].route_id
    for route_id in top_route_ids:
        route = df_long[df_long["route_id"] == route_id].sort_values("position_time")
        ax.plot(route["lon"], route["lat"], transform=ccrs.PlateCarree())

    ax.set_extent([-3, 10, 55, 65])
    ax.set_title(month)

plt.tight_layout()
plt.show()

In [ ]:
import cartopy.crs as ccrs

fig = plt.figure(figsize=(20, 25))
for i, (date, data) in enumerate(
    df_round[df_round["t_round"].dt.year >= 2020].groupby("t_round")
):
    # print(date, date.strftime('%B %Y'))
    print(date, date)
    ax = plt.subplot(5, 7, i + 1, projection=ccrs.epsg("3035"))
    ax.stock_img()
    # ax.scatter(data.lon, data.lat, c=data.roundweight, transform=ccrs.PlateCarree())
    ax.scatter(
        data.x_round, data.y_round, c=data.roundweight, s=5, vmin=vmin, vmax=vmax
    )
    # ax.set_extent([-3, 10, 55, 65])
    ax.set_extent([x_min, x_max, y_min, y_max], crs=ccrs.epsg("3035"))
    # ax.set_title(date.strftime('%B %Y'))
    ax.set_title(date)
    # ax.set_extent([-3, 10, 55, 65])

plt.tight_layout()
plt.show()

In [ ]:
import cartopy.crs as ccrs

data = df[df["position_time"].between("2015-09-01", "2022-11-01")]
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.epsg("3035"))
ax.stock_img()
# ax.scatter(data.lon, data.lat, c=data.roundweight, transform=ccrs.PlateCarree())
ax.scatter(data.x_round, data.y_round, c=data.roundweight)
ax.set_extent([-3, 10, 55, 65])
# ax.set_extent([-3, 10, 55, 65])
plt.show()

In [ ]:
import cartopy.crs as ccrs

data = df_round[df_round["t_round"].between("2015-09-01", "2022-11-01")]
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.epsg("3035"))
ax.stock_img()
ax.scatter(data.x_round, data.y_round, c=data.roundweight)
ax.set_extent([-3, 10, 55, 65])
plt.show()

In [ ]:
# use grid search cross-validation to optimize the bandwidth
params = {"bandwidth": np.logspace(-1, 1, 20)}
grid = GridSearchCV(KernelDensity(), params)
grid.fit(data)

In [ ]:
print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))

# use the best estimator to compute the kernel density estimate
kde = grid.best_estimator_

# sample 44 new points from the data
new_data = kde.sample(44, random_state=0)
new_data = pca.inverse_transform(new_data)

# turn data into a 4x11 grid
new_data = new_data.reshape((4, 11, -1))
real_data = digits.data[:44].reshape((4, 11, -1))

# plot real digits and resampled digits
fig, ax = plt.subplots(9, 11, subplot_kw=dict(xticks=[], yticks=[]))
for j in range(11):
    ax[4, j].set_visible(False)
    for i in range(4):
        im = ax[i, j].imshow(
            real_data[i, j].reshape((8, 8)), cmap=plt.cm.binary, interpolation="nearest"
        )
        im.set_clim(0, 16)
        im = ax[i + 5, j].imshow(
            new_data[i, j].reshape((8, 8)), cmap=plt.cm.binary, interpolation="nearest"
        )
        im.set_clim(0, 16)

ax[0, 5].set_title("Selection from the input data")
ax[5, 5].set_title('"New" digits drawn from the kernel density model')

plt.show()